In [13]:
#this script pivots the currently existing rankings sheet and pivots it from wide to long.

#imports packages
import pandas as pd
import numpy as np
import re

In [14]:
#function used to extract just the portion of the result string indicating the result. Neccessary as the result/points columns are concatenated to get a single column for each tournament
#for pivoting purposes.
def parse_raw_result(raw_result):
    if raw_result=='':
        return ''
    elif ('"') in raw_result:
        num_pattern=r"(\"\d-\d\")"
        return re.findall(num_pattern,raw_result)[0]
    else:
        letter_pattern=r"([A-Z\*\^]+)"
        return re.findall(letter_pattern,raw_result)[0]

In [15]:
#result raw includes place and points (eg C**21.0), this gets only the points
def extract(result,result_raw):
    return result_raw[len(result):]

In [16]:
#sets path
path='/Users/nickhansen/Anaconda/Tabroom/pivot_existing/rankings_official.xlsx'
#sets sheet
sheet='Team'
#reads excel
df=pd.read_excel(path,sheet_name=sheet,dtype='str')
#index where, if given this in iloc, will return name of first competition
first_tournament_index=7

In [17]:
#grabs a list of all tournaments, excluding team info
comp_df_only=df.iloc[:,first_tournament_index:]
#list comprehension used to get tournament names only, excluding Unnamed columns
tournaments=[a for a in comp_df_only.columns.tolist() if type(a)==str and a[:7]!='Unnamed'][:-7]
#gets indices from dataframe for each tournament
indices=[df.columns.get_loc(c) for c in tournaments if c in df]
#gets dataframe that includes only team info
team_info_only=df.iloc[:,0:first_tournament_index]
#gets team info cols
cols=list(team_info_only)+list(tournaments)
#convers all datatypes to string
df=df.astype(str)


In [18]:
#concatenates result and points column
for i in range(len(tournaments)):
    df[tournaments[i]]=df[tournaments[i]]+df.iloc[:,indices[i]+1]
#gets only columns
df=df[cols]
#fills with na
df=df.replace('nannan',np.nan)
#pivots wide to long
df=pd.melt(df,id_vars=list(team_info_only.columns),
        value_vars=tournaments,value_name='result',var_name='tournament')

#saves result as result_raw, since result will be cleaned
df['result_raw']=df['result']
#grabs only needed columns
df=df[['Rank','School','Partner 1','Partner 2','Points','Unique Team Points','Region','tournament','result_raw']]
#grabs only rwos where there's a result
df=df[df['result_raw'].notnull()]
#fills
df['result_raw']=df['result_raw'].replace('nan','')

In [19]:
#applies function referenced earlier
df['result']=df['result_raw'].apply(parse_raw_result)
#fills na
df.result = df.result.fillna('')
#applies function referenced earlier
df['points']=df.apply(lambda x: extract(x['result'],x['result_raw']),axis=1)


In [20]:
#renames column
df.columns=['Rank', 'School', 'Partner 1', 'Partner 2', 'points_old',
       'unique_team_points_old', 'Region', 'tournament', 'result_raw', 'result',
       'points']
df=df.iloc[:,1:]


In [21]:
#generates team name 
df['Team Name']=df['School']+' '+df['Partner 1']+' & '+df['Partner 2']


In [22]:
#columns and filters
df=df[['School', 'Partner 1', 'Partner 2', 'tournament','result','points','Region', 'result_raw',
   'Team Name']]
df=df[df['result_raw']!='']
df

,School,Partner 1,Partner 2,tournament,result,points,Region,result_raw,Team Name
0,Nueva,Kan,Sachdev,Notre Dame,C**,21.0,CA1,C**21.0,Nueva Kan & Sachdev
4,Nueva,Wee,Srihari,Notre Dame,O,8.0,CA1,O8.0,Nueva Wee & Srihari
5,Nueva,Patel,Ramanathan,Notre Dame,C*,23.0,CA1,C*23.0,Nueva Patel & Ramanathan
7,Los Altos,Kadiyala,Lingo,Notre Dame,Q*,10.0,CA1,Q*10.0,Los Altos Kadiyala & Lingo
8,Washington,Jain,Gandhi,Notre Dame,O,8.0,CA1,O8.0,Washington Jain & Gandhi
...,...,...,...,...,...,...,...,...,...
22504,Lowell,Blacker,Goggin,GGSA 1,"""2-1""",2.0,CA1,"""2-1""2.0",Lowell Blacker & Goggin
22506,Monte Vista,Prakash,Yuan,GGSA 1,"""2-1""",2.0,CA1,"""2-1""2.0",Monte Vista Prakash & Yuan
23860,Turlock,Sanchez,Sorisho,YFL 1,"""3-0""",5.0,CA2,"""3-0""5.0",Turlock Sanchez & Sorisho
23957,Modesto,Rai,Vijaykumar,YFL 1,"""2-1""",2.0,CA2,"""2-1""2.0",Modesto Rai & Vijaykumar


In [23]:
#exports to csv
df.to_csv('./pivot_export.csv')